---
**License**

 NumCosmoCCLTest

 Mon Jun 01 09:20:00 2020\
 Copyright  2020\
 Sandro Dias Pinto Vitenti <vitenti@uel.br>

---
---

 NumCosmoCCLTest\
 Copyright (C) 2020 Sandro Dias Pinto Vitenti <vitenti@uel.br>

 numcosmo is free software: you can redistribute it and/or modify it
 under the terms of the GNU General Public License as published by the
 Free Software Foundation, either version 3 of the License, or
 (at your option) any later version.

 numcosmo is distributed in the hope that it will be useful, but
 WITHOUT ANY WARRANTY; without even the implied warranty of
 MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.
 See the GNU General Public License for more details.

 You should have received a copy of the GNU General Public License along
 with this program.  If not, see <http://www.gnu.org/licenses/>.
 
---

# The distance comoving

This notebook calculates the comoving distances using the Numcosmo and CCL libraries, and then it compares the results between themselves, plotting the compare and the relative distance of their results in graphics.


### Loading NumCosmo and CCL

The first step is to load both NumCosmo and CCL libraries. We also load some Python packages.

In [ ]:
try:
  import gi
  gi.require_version('NumCosmo', '1.0')
  gi.require_version('NumCosmoMath', '1.0')
except:
  pass

from gi.repository import GObject
from gi.repository import NumCosmo as Nc
from gi.repository import NumCosmoMath as Ncm

import pyccl as ccl

import sys
import numpy as np
import math
import matplotlib.pyplot as plt


### Initializing the NumCosmo library:

In [ ]:
Ncm.cfg_init ()
Ncm.cfg_set_log_handler (lambda msg: sys.stdout.write (msg) and sys.stdout.flush ())

### The function of building comparison graphics:
Assembles two comparison graphs, where the graph above presents the result curves of both libraries, while the graph below presents the difference relative distance curve between the results found using the same values in the abscissas axis.                                                                                               

In [ ]:
import pylab as plt
def compare(x, y_ccl, y_nc, x_name='x', y_name='func', subplots_pars={'figsize':(12, 6)}, xscale = 'linear', yscale = 'log'):
    ccl_name, nc_name = '%s_{clmm}'%y_name, '%s_{nc}'%y_name
    
    x     = np.array (x)
    y_ccl = np.array (y_ccl)
    y_nc  = np.array (y_nc)
    diff  = np.zeros_like (y_ccl)
    
    non_zind = np.where (y_ccl != 0.0)[0]
    zind     = np.where (y_ccl == 0.0)[0]
    diff[non_zind] = y_nc[non_zind]/y_ccl[non_zind] - 1.0
    diff[zind]     = y_nc[zind] - y_ccl[zind]
    print('[%10s]: rel diff min: %e\trel diff max: %e'%(y_name, min(abs(diff)), max(abs(diff))))
        
    fig, axs = plt.subplots(2, sharex=True, **subplots_pars)
    fig.subplots_adjust(hspace=0)
    
    axs[0].plot(x, y_ccl, label='ccl', lw=3)
    axs[0].plot(x, y_nc, label='nc')
    axs[1].plot(x, np.abs (diff), c='r')
    #axs[1].axhline(0, ls=':', c='0', lw=.5)
    axs[1].set_xscale (xscale)
    axs[1].set_yscale (yscale)

    axs[0].legend()
    axs[0].set_ylabel('$%s$'%y_name)
    axs[1].set_xlabel('$%s$'%x_name)
    axs[1].set_ylabel('$%s/%s-1$'%(nc_name, ccl_name))
    
    return fig, axs

### The fixed cosmological parameters using the cosmology object:

* The cold dark matter density fraction $\Omega_c$ ;
* The baryonic matter density fraction $\Omega_b$ ;
* The curvature density fraction $\Omega_k$;
* The hubble constant divided by 100 km/s/Mpc and unitless $h$ ;
* The effective number of massless neutrinos present $N_{\nu}$ ;
* The primordial scalar perturbation spectral index $n_s$ ;
* The power spectrum normalization $A_s$ ;

In [ ]:
Omega_c = 0.25   # The cold dark matter density fraction.

Omega_b = 0.05   # The baryonic matter density fraction.

Omega_k = 0.0    # The curvature density fraction.
        
h       = 0.7    # The hubble constant divided by 100 km/s/Mpc; unitless.

A_s     = 2.1e-9 # The power spectrum normalization.

n_s     = 0.96   # The primordial scalar perturbation spectral index.

Neff    = 0.0    # The effective number of massless neutrinos present.

### The arbitrary cosmological parameters using the cosmology object:

* The dark energy fraction $\Omega_{\Lambda}$ ; 
* The first order term of dark energy equation of state $w_0$.
* The second order term of dark energy equation of state $w_a$ . 

In [ ]:
Omega_v_vals = np.array([0.7, 0.7, 0.7, 0.65, 0.75])    # The values of Omega_Lamda.

w0_vals      = np.array([-1.0, -0.9, -0.9, -0.9, -0.9]) # The first order term of dark energy equation of state.

wa_vals      = np.array([0.0, 0.0, 0.1, 0.1, 0.1])      # The second order term of dark energy equation of state.

In [ ]:
#  Neff_mnu = 3.0<-- Doesn't seem to be used
#  mu_0    = 0.1 <-- Doesn't seem to be used
#  sigma_0 = 0.1 <-- Doesn't seem to be used

#mnu = [  [0.04, 0., 0.], <-- Doesn't seem to be used
#         [0.05, 0.01, 0.],
#         [0.05, 0., 0.],
#         [0.03, 0.02, 0.]]

### The compare distances function
The Function below calculates three types of comparisons and runs the compare function to build the graphs related to each calculation by varying the arbitrary cosmological parameters described above and comparing the calculation of

1. The baryonic matter density fraction at a redshift different than z=0.

2. The ratio of Hubble constant at the scale factor over Hubble constant today.

3. The calculation of the comoving radial distance.

In [ ]:
def compare_distances(z, Omega_v, w0, wa):
    """
    Compare distances calculated by pyccl with the distances in the benchmark
    file.
    This test is only valid when radiation is explicitly set to 0.
    """
    # Set Omega_K in a consistent way
    Omega_k = 1.0 - Omega_c - Omega_b - Omega_v

    ccl_cosmo = ccl.Cosmology(
        Omega_c=Omega_c, Omega_b=Omega_b, Neff=Neff,
        h=h, A_s=A_s, n_s=n_s, Omega_k=Omega_k,
        w0=w0, wa=wa, Omega_g=0)
    
    ccl_cosmo.cosmo.gsl_params.INTEGRATION_EPSREL = 1.0e-13 # Doesn't seem to work
    ccl_cosmo.cosmo.gsl_params.A_SPLINE_NLOG      = 100000  # Doesn't seem to work
    ccl_cosmo.cosmo.gsl_params.A_SPLINE_NA        = 100000  # Doesn't seem to work
        
    cosmo = Nc.HICosmo.new_from_name (Nc.HICosmo, "NcHICosmoDECpl{'massnu-length':<0>}")
    cosmo.omega_x2omega_k ()
    cosmo.param_set_by_name ("H0",        h*100)
    cosmo.param_set_by_name ("Omegak",    Omega_k)
    cosmo.param_set_by_name ("w0",        w0)
    cosmo.param_set_by_name ("w1",        wa)
    cosmo.param_set_by_name ("Omegab",    Omega_b)
    cosmo.param_set_by_name ("Omegac",    Omega_c)
    cosmo.param_set_by_name ("ENnu",      Neff)
    cosmo.param_set_by_name ("Tgamma0",   0.0)
    
    hiprim = Nc.HIPrimPowerLaw.new ()
    hiprim.param_set_by_name ("ln10e10ASA", math.log (1.0e10 * A_s))
    hiprim.param_set_by_name ("n_SA",       n_s)
    
    cosmo.add_submodel (hiprim)
    
    dist = Nc.Distance.new (5.0)
    dist.prepare (cosmo)
    
    print (dist.comoving_distance_spline.props.reltol)
    print (dist.comoving_distance_spline.props.abstol)
    print (dist.comoving_distance_spline.peek_spline().get_len())
    
    a = 1. / (1. + z)    

    ccl_Og = ccl.omega_x (ccl_cosmo, a, 'matter')
    nc_Og  = [cosmo.E2Omega_m (z_i) / cosmo.E2 (z_i) for z_i in z]
    
    compare (z, ccl_Og, nc_Og, x_name = 'z', y_name = '\Omega_m{}')
    
    ccl_E = ccl.h_over_h0 (ccl_cosmo, a)
    nc_E  = [cosmo.E (z_i) for z_i in z]
    
    compare (z, ccl_E, nc_E, x_name = 'z', y_name = 'E')
    
    ccl_chi = ccl.comoving_radial_distance(ccl_cosmo, a)
    RH_Mpc  = cosmo.RH_Mpc ()
    nc_chi  = [dist.comoving (cosmo, z_i) * RH_Mpc for z_i in z]
    
    compare (z, ccl_chi, nc_chi, x_name = 'z', y_name = 'D')

In [ ]:
z = np.linspace (0.0, 5.0, 10000) #Using 10000 redshift values between 0 and 5 with even spacing between values to compare.

In [ ]:
compare_distances(z, Omega_v_vals[0], w0_vals[0], wa_vals[0])  #Omega_v = 0.7 , w0 = -1.0 , wa = 0.0

In [ ]:
compare_distances(z, Omega_v_vals[1], w0_vals[1], wa_vals[1])  #Omega_v = 0.7 , w0 = -0.9 , wa = 0.0

In [ ]:
compare_distances(z, Omega_v_vals[2], w0_vals[2], wa_vals[2])  #Omega_v = 0.7 , w0 = -0.9 , wa = 0.1

In [ ]:
compare_distances(z, Omega_v_vals[3], w0_vals[3], wa_vals[3])  #Omega_v = 0.65 , w0 = -0.9 , wa = 0.1

In [ ]:
compare_distances(z, Omega_v_vals[4], w0_vals[4], wa_vals[4])  #Omega_v = 0.75 , w0 = -0.9 , wa = 0.1